# Original table analysis

#### Which sub probabilities are included in the original table?

In [ ]:
setting = dict(
Qe_values = [0x5601,0x3401,0x1801,0x0ac1,0x0521,0x0221,0x5601,0x5401,0x4801,0x3801,
  0x3001,0x2401,0x1c01,0x1601,0x5601,0x54ff,0x5401,0x527d,0x5101,0x4c5f,
  0x4801,0x3f80,0x3801,0x35f7,0x3401,0x31f6,0x3001,0x2801,0x2401,0x2201,
  0x1c01,0x1801,0x1601,0x1401,0x1201,0x1101,0x0ac1,0x09c1,0x08a1,0x0521,
  0x0441,0x02a1,0x0221,0x0141,0x0111,0x0085,0x0049,0x0025,0x0015,0x0009,
  0x0005,0x0001,0x5601],

Qe_switch = [1,0,0,0,0,0,1,0,0,0,
  0,0,0,0,1,0,0,0,0,0,
  0,0,0,0,0,0,0,0,0,0,
  0,0,0,0,0,0,0,0,0,0,
  0,0,0,0,0,0,0,0,0,0,
  0,0,0],

Qe_MPS = [1,2,3,4,5,44,7,8,9,10,
  11,12,13,35,15,16,17,18,19,20,
  21,22,23,24,25,26,27,28,29,30,
  31,32,33,34,35,36,37,38,39,40,
  41,42,43,44,45,45,47,48,49,50,
  51,51,52],

Qe_LPS = [  1,6,9,12,35,39,6,14,14,14,
  20,22,25,27,14,14,14,15,16,17,
  18,19,20,21,22,23,24,25,26,27,
  28,29,30,31,32,33,34,35,36,37,
  38,39,40,41,42,43,44,45,46,47,
  48,49,52],)

# Probability that MPS occurs = 1 - Qe_decimal

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df = pd.DataFrame(setting)
df['Qe_decimal'] = df.loc[:,'Qe_values'] * 1.5 / (1 << 16)

In [ ]:
df

- The table consists of several different probability declines
- Starting point of these intervals are where Qe_LPS > Index **?**

In [ ]:
starting_point_of_new_intervals = np.append([0], df.Qe_LPS[df.Qe_LPS >= df.index].unique())
backward_point_of_old_intervals = [x for x in range(53) if x not in starting_point_of_new_intervals]

In [ ]:
def follow_mps(df, start):
    ix = start
    probabilities = []
    while True:
        prob = df.loc[ix, 'Qe_decimal']
        probabilities.append(prob)
        if df.loc[ix,'Qe_MPS'] == ix:
            break
        ix = df.loc[ix,'Qe_MPS']
    return np.array(probabilities)

def follow_lps(df, start):
    ix = start
    probabilities = []
    while True:
        prob = df.loc[ix, 'Qe_decimal']
        probabilities.append(prob)
        if df.loc[ix,'Qe_LPS'] == ix:
            break
        ix = df.loc[ix,'Qe_LPS']
    return np.array(probabilities)

In [ ]:
def append(to, what, how_often, front=True):
    tmp = [what] * how_often
    if front:
        return np.append(tmp, to)
    return np.append(to, tmp)

In [ ]:
probabilities = {'S'+str(s): follow_mps(df, s) for s in starting_point_of_new_intervals}
adjusted_prob = {s: append(k, None, 32 - len(k), False) for s,k in probabilities.items()}

In [ ]:
pd.DataFrame(adjusted_prob).plot(figsize=(16.53,11.69), marker='x');
plt.tight_layout()
plt.savefig('mps.a3.svg')

In [ ]:
pd.DataFrame(adjusted_prob).plot(figsize=(11.69,8.27),subplots=True, sharey=True);

In [ ]:
probabilities = {'S'+str(s): follow_lps(df, s) for s in backward_point_of_old_intervals}
adjusted_prob = {s: append(k, None, 32 - len(k), False) for s,k in probabilities.items()}

In [ ]:
pd.DataFrame(adjusted_prob).plot(figsize=(16.53,11.69), marker='x');
plt.tight_layout()
plt.savefig('lps.a3.svg')

In [ ]:
pd.DataFrame(adjusted_prob).plot(figsize=(8.27,11.69),subplots=True, sharey=True);

Preprocessing is done. The plot and changes are printed.